# Brier Score Calculator

This notebook calculates the Brier Score using the function from sklearn.

In [1]:
!hostname
# Must be used on regular nodes with a lot of ram

v320-010.ls6.tacc.utexas.edu


In [2]:
import pandas as pd
from joblib import Parallel, delayed
from sklearn.metrics import brier_score_loss
from xskillscore import brier_score
import os

WORK = os.environ.get('WORK')
SCRATCH = os.environ.get('SCRATCH')

In [3]:
import glob
era_paths = {month:sorted(list(glob.glob(f'{SCRATCH}/hindcast/xarray/era_stage2/era_rain_*-{str(month).zfill(2)}.nc',recursive=True))) for month in range(1,13)}
hc_paths = {month:sorted(list(glob.glob(f'{SCRATCH}/hindcast/xarray/stage2/surface_variables_rain_*-{str(month).zfill(2)}.nc',recursive=True))) for month in range(1,13)}

In [41]:
import xarray as xr
from rainproj.brier_score import calc_prob


result = {}
for month in range(1,13):
    ds_era = xr.open_mfdataset(era_paths[month],engine='netcdf4').pipe(calc_prob,'total_precipitation').rename({'time':'date'})
    ds_hc = xr.open_mfdataset(hc_paths[month],engine='netcdf4').pipe(calc_prob,'tp06')

    for lead in range(0,8):
        print(f'{month} {lead}')

        result[month] = {f'{lead}D':brier_score(ds_era.sel(date=ds_hc['date'])['>2.5mm'].astype(float),ds_hc.sel(lead_time=f'{lead}D').reset_coords(names="lead_time", drop=True)['>2.5mm'].astype(float)).compute()}


1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7


In [55]:
ds_era_all = xr.open_mfdataset(glob.glob(f'{SCRATCH}/hindcast/xarray/era_stage2/era_rain_*.nc'),engine='netcdf4')
ds_hc_all = xr.open_mfdataset(glob.glob(f'{SCRATCH}/hindcast/xarray/stage2/surface_variables_rain_*.nc'),engine='netcdf4')

In [71]:
ds_era_all2 = ds_era_all.pipe(calc_prob,'total_precipitation').rename({'time':'date'}).sel(date=ds_hc_all['date'])
ds_hc_all2 = ds_hc_all.pipe(calc_prob,'tp06')

In [88]:
ds_era_all2['month'] = ds_era_all2['date'].dt.month
ds_hc_all2['month'] = ds_hc_all2['date'].dt.month
ds_era_all2 = ds_era_all2.set_coords('month')
ds_hc_all2 = ds_hc_all2.set_coords('month')

In [89]:
ds_hc_all2

<xarray.Dataset> Size: 19GB
Dimensions:    (date: 16802, lead_time: 8, lat: 99, lon: 233)
Coordinates:
  * lat        (lat) float64 792B 25.0 25.25 25.5 25.75 ... 49.0 49.25 49.5
  * lon        (lon) float64 2kB 235.0 235.2 235.5 235.8 ... 292.5 292.8 293.0
  * lead_time  (lead_time) timedelta64[ns] 64B 0 days 1 days ... 6 days 7 days
  * date       (date) datetime64[ns] 134kB 1979-01-01 1979-01-02 ... 2024-12-31
    month      (date) int64 134kB 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12
Data variables:
    tp06       (date, lead_time, lat, lon) float32 12GB dask.array<chunksize=(31, 8, 99, 233), meta=np.ndarray>
    >2.5mm     (date, lead_time, lat, lon) bool 3GB dask.array<chunksize=(31, 8, 99, 233), meta=np.ndarray>
    >25mm      (date, lead_time, lat, lon) bool 3GB dask.array<chunksize=(31, 8, 99, 233), meta=np.ndarray>

In [72]:
ds_era_all2.sel(date=(ds_era_all2['date'].dt.month==2))

<xarray.Dataset> Size: 180MB
Dimensions:              (date: 1300, lat: 99, lon: 233)
Coordinates:
  * lat                  (lat) float32 396B 25.0 25.25 25.5 ... 49.0 49.25 49.5
  * lon                  (lon) float32 932B 235.0 235.2 235.5 ... 292.8 293.0
  * date                 (date) datetime64[ns] 10kB 1979-02-01 ... 2024-02-29
Data variables:
    total_precipitation  (date, lat, lon) float32 120MB dask.array<chunksize=(29, 99, 233), meta=np.ndarray>
    >2.5mm               (date, lat, lon) bool 30MB dask.array<chunksize=(29, 99, 233), meta=np.ndarray>
    >25mm                (date, lat, lon) bool 30MB dask.array<chunksize=(29, 99, 233), meta=np.ndarray>

In [76]:
ds_hc_all2.sel(date=(ds_hc_all2['date'].dt.month==month),lead_time=f'{lead}D').reset_coords(names="lead_time", drop=True)['>2.5mm'].astype(float)

<xarray.DataArray '>2.5mm' (date: 1426, lat: 99, lon: 233)> Size: 263MB
dask.array<astype, shape=(1426, 99, 233), dtype=float64, chunksize=(30, 99, 233), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 792B 25.0 25.25 25.5 25.75 ... 48.75 49.0 49.25 49.5
  * lon      (lon) float64 2kB 235.0 235.2 235.5 235.8 ... 292.5 292.8 293.0
  * date     (date) datetime64[ns] 11kB 1979-01-01 1979-01-02 ... 2024-01-31

In [90]:
brier_score(ds_era_all2['>2.5mm'].astype(float),ds_hc_all2.sel(lead_time=f'0D').reset_coords(names="lead_time", drop=True)['>2.5mm'].astype(float),member_dim="month").compute()

<xarray.DataArray '>2.5mm' ()> Size: 8B
array(0.06900645)

In [ ]:
result = {}
for month in range(1,13):
    ds_era = ds_era_all2.sel(date=(ds_era_all2['date'].dt.month==month)).compute()
    ds_hc = ds_hc_all2.sel(date=(ds_hc_all2['date'].dt.month==month)).compute()
    result_month={}
    for lead in range(0,8):
        print(f'{month} {lead}')

        result_month[f'{lead}D'] = brier_score(ds_era['>2.5mm'].astype(float),ds_hc.sel(lead_time=f'{lead}D').reset_coords(names="lead_time", drop=True)['>2.5mm'].astype(float)).compute().values
    result[month] = result_month

1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7


In [ ]:
res

In [101]:
result2 = {k:{_k:_v.values for _k,_v in v.items()} for k,v in result.items() }

In [105]:
import pandas as pd

df = pd.DataFrame.from_dict(result2,orient='index').astype(float)

In [112]:
import pandas as pd

pd.options.plotting.backend = 'plotly'
fig = df.plot()
fig.update_layout(autosize=False,template='simple_white')
fig.show()

In [ ]:
result = {}
for month in range(1,13):
    ds_era = xr.open_mfdataset(era_paths[month],engine='netcdf4').pipe(calc_prob,'total_precipitation').rename({'time':'date'})
    ds_hc = xr.open_mfdataset(hc_paths[month],engine='netcdf4').pipe(calc_prob,'tp06')

    for lead in range(0,8):
        print(f'{month} {lead}')

        result[month] = {f'{lead}D':brier_score(ds_era.sel(date=ds_hc['date'])['>2.5mm'].astype(float),ds_hc.sel(lead_time=f'{lead}D').reset_coords(names="lead_time", drop=True)['>2.5mm'].astype(float)).compute()}


In [38]:
brier_score(ds_era.sel(date=ds_hc['date'])['>2.5mm'].astype(float),ds_hc.sel(lead_time='1D').reset_coords(names="lead_time", drop=True)['>2.5mm'].astype(float)).compute()

<xarray.DataArray '>2.5mm' ()> Size: 8B
array(0.19654025)

In [ ]:
ds_hc.sel(lead_time='1D')['>2.5mm']

<xarray.DataArray '>2.5mm' (date: 1426, lat: 99, lon: 233)> Size: 33MB
dask.array<getitem, shape=(1426, 99, 233), dtype=bool, chunksize=(31, 99, 233), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float64 792B 25.0 25.25 25.5 25.75 ... 49.0 49.25 49.5
  * lon        (lon) float64 2kB 235.0 235.2 235.5 235.8 ... 292.5 292.8 293.0
    lead_time  timedelta64[ns] 8B 1 days
  * date       (date) datetime64[ns] 11kB 1979-01-01 1979-01-02 ... 2024-01-31

In [ ]:
lead_day = '1D'
month = 12

ds_hc.sel(lead_time='1D')['>2.5mm']

<xarray.Dataset> Size: 197MB
Dimensions:    (date: 1426, lat: 99, lon: 233)
Coordinates:
  * lat        (lat) float64 792B 25.0 25.25 25.5 25.75 ... 49.0 49.25 49.5
  * lon        (lon) float64 2kB 235.0 235.2 235.5 235.8 ... 292.5 292.8 293.0
    lead_time  timedelta64[ns] 8B 1 days
  * date       (date) datetime64[ns] 11kB 1979-01-01 1979-01-02 ... 2024-01-31
Data variables:
    tp06       (date, lat, lon) float32 132MB dask.array<chunksize=(31, 99, 233), meta=np.ndarray>
    >2.5mm     (date, lat, lon) bool 33MB dask.array<chunksize=(31, 99, 233), meta=np.ndarray>
    >25mm      (date, lat, lon) bool 33MB dask.array<chunksize=(31, 99, 233), meta=np.ndarray>
    month      (date) int64 11kB 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    day        (date) int64 11kB 1 2 3 4 5 6 7 8 9 ... 24 25 26 27 28 29 30 31

In [7]:
ds_era

<xarray.Dataset> Size: 198MB
Dimensions:              (date: 1427, lat: 99, lon: 233)
Coordinates:
  * lat                  (lat) float32 396B 25.0 25.25 25.5 ... 49.0 49.25 49.5
  * lon                  (lon) float32 932B 235.0 235.2 235.5 ... 292.8 293.0
  * date                 (date) datetime64[ns] 11kB 1979-01-01 ... 2025-01-01
Data variables:
    total_precipitation  (date, lat, lon) float32 132MB dask.array<chunksize=(31, 99, 233), meta=np.ndarray>
    >2.5mm               (date, lat, lon) bool 33MB dask.array<chunksize=(31, 99, 233), meta=np.ndarray>
    >25mm                (date, lat, lon) bool 33MB dask.array<chunksize=(31, 99, 233), meta=np.ndarray>
    month                (date) int64 11kB 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    day                  (date) int64 11kB 1 2 3 4 5 6 7 ... 26 27 28 29 30 31 1

In [46]:
[group for group in ds.groupby(['month','day'])][0][1]

<xarray.Dataset> Size: 7MB
Dimensions:              (time: 47, lat: 99, lon: 233)
Coordinates:
  * lat                  (lat) float32 396B 25.0 25.25 25.5 ... 49.0 49.25 49.5
  * lon                  (lon) float32 932B 235.0 235.2 235.5 ... 292.8 293.0
  * time                 (time) datetime64[ns] 376B 1979-01-01 ... 2025-01-01
Data variables:
    total_precipitation  (time, lat, lon) float32 4MB dask.array<chunksize=(30, 99, 233), meta=np.ndarray>
    >2.5mm               (time, lat, lon) bool 1MB dask.array<chunksize=(30, 99, 233), meta=np.ndarray>
    >25mm                (time, lat, lon) bool 1MB dask.array<chunksize=(30, 99, 233), meta=np.ndarray>
    month                (time) int64 376B 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    day                  (time) int64 376B 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1

In [ ]:

ds['>2.5mm climate'] = ds.groupby(['month','day'])['>2.5mm'].transform('mean')
ds['>25mm climate'] = ds.groupby(['month','day'])['>25mm'].transform('mean')

KeyError: 1

In [25]:
ds['time'].dt.day

<xarray.DataArray 'day' (time: 1427)> Size: 11kB
array([ 1,  2,  3, ..., 30, 31,  1], shape=(1427,))
Coordinates:
  * time     (time) datetime64[ns] 11kB 1979-01-01 1979-01-02 ... 2025-01-01

In [19]:
ds['>2.5mm']=ds['total_precipitation']
ds

<xarray.Dataset> Size: 263MB
Dimensions:              (time: 1427, lat: 99, lon: 233)
Coordinates:
  * lat                  (lat) float32 396B 25.0 25.25 25.5 ... 49.0 49.25 49.5
  * lon                  (lon) float32 932B 235.0 235.2 235.5 ... 292.8 293.0
  * time                 (time) datetime64[ns] 11kB 1979-01-01 ... 2025-01-01
Data variables:
    total_precipitation  (time, lat, lon) float32 132MB dask.array<chunksize=(31, 99, 233), meta=np.ndarray>
    >2.5mm               (time, lat, lon) float32 132MB dask.array<chunksize=(31, 99, 233), meta=np.ndarray>

In [20]:
ds.index.get_level_values('time').month

AttributeError: 'Dataset' object has no attribute 'index'

In [ ]:
from rainproj.brier_score import process_all
test2 = process_all(era_paths=era_paths,hc_paths=hc_paths,xskill=False)

load done
start tasks


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame.from_dict({(month,lead):metrics for month,lead_list in test2.items() for lead,metrics in lead_list.items()},orient='index').sort_index()
df.index.set_names(['Month','Lead Day'],inplace=True)
df.to_parquet(f'{WORK}/rainproj/data/br_loss_score.parquet')

## Unused Code
***

In [ ]:
raise ValueError

# AORC Addendum

***

In [ ]:
hc_df

precipitation tp06 [m]  >2.5mm  >25mm
date       lat  lon   lead_time                                       
1979-01-01 25.0 235.0 0 days                   0.000096   False  False
                      1 days                   0.004315    True  False
                      2 days                   0.004706    True  False
                      3 days                   0.003211    True  False
                      4 days                   0.001623   False  False
...                                                 ...     ...    ...
2024-01-31 49.5 293.0 3 days                   0.001326   False  False
                      4 days                   0.000955   False  False
                      5 days                   0.009829    True  False
                      6 days                   0.001459   False  False
                      7 days                   0.000030   False  False

[263148336 rows x 3 columns]

In [ ]:
era_df.loc[hc_df.loc[idx[:,:,:,pd.Timedelta('7D')]].index]

total_precipitation  >2.5mm  >25mm
time       lat  lon                                       
1979-01-01 25.0 235.00         1.159403e-04   False  False
                235.25         4.931912e-05   False  False
                235.50         2.076663e-05   False  False
                235.75         4.328787e-06   False  False
                236.00         8.661300e-07   False  False
...                                     ...     ...    ...
2024-01-31 49.5 292.00         2.423152e-04   False  False
                292.25         2.007447e-04   False  False
                292.50         1.713447e-04   False  False
                292.75         1.632310e-04   False  False
                293.00         1.571514e-04   False  False

[32893542 rows x 3 columns]

In [ ]:
idx = pd.IndexSlice

hc_df.loc[idx[:,:,:,pd.Timedelta('7D')]].index

precipitation tp06 [m]  >2.5mm  >25mm
date       lat  lon                                          
1979-01-01 25.0 235.00                0.000064   False  False
                235.25                0.000072   False  False
                235.50                0.000075   False  False
                235.75                0.000079   False  False
                236.00                0.000072   False  False
...                                        ...     ...    ...
2024-01-31 49.5 292.00               -0.000054   False  False
                292.25               -0.000054   False  False
                292.50               -0.000016   False  False
                292.75                0.000022   False  False
                293.00                0.000030   False  False

[32893542 rows x 3 columns]

In [ ]:
common_idx = hc_df.index.droplevel('lead_time').intersection(era_df.index)

In [ ]:
# lead='0D'

# era_1980 = {mo:pd.read_parquet(era_paths[mo]) for mo in range(1,13)}
# hc_1980_0L = {mo:pd.read_parquet(hc_paths[mo]).loc[idx[:,:,:,pd.Timedelta(lead)]] for mo in range(1,13)}

In [ ]:
lead_days[lead_days<=pd.Timedelta('6D')]

TimedeltaIndex(['0 days', '1 days', '2 days', '3 days', '4 days', '5 days',
                '6 days'],
               dtype='timedelta64[ns]', name='lead_time', freq=None)

In [ ]:
hc_df.index.get_level_values('lead_time').max()

Timedelta('7 days 00:00:00')

In [ ]:
idx = pd.IndexSlice

hc_df.loc[idx[:,:,:,pd.Timedelta('7D')]]

precipitation tp06 [m]  >2.5mm  >25mm
date       lat  lon                                          
1979-01-01 25.0 235.00                0.000064   False  False
                235.25                0.000072   False  False
                235.50                0.000075   False  False
                235.75                0.000079   False  False
                236.00                0.000072   False  False
...                                        ...     ...    ...
2024-01-31 49.5 292.00               -0.000054   False  False
                292.25               -0.000054   False  False
                292.50               -0.000016   False  False
                292.75                0.000022   False  False
                293.00                0.000030   False  False

[32893542 rows x 3 columns]

In [ ]:
era_df

total_precipitation  >2.5mm  >25mm
time       lat  lon                                       
1979-01-01 25.0 235.00         1.159403e-04   False  False
                235.25         4.931912e-05   False  False
                235.50         2.076663e-05   False  False
                235.75         4.328787e-06   False  False
                236.00         8.661300e-07   False  False
...                                     ...     ...    ...
2025-01-01 49.5 292.00         3.803242e-04   False  False
                292.25         4.865769e-04   False  False
                292.50         5.792510e-04   False  False
                292.75         5.113427e-04   False  False
                293.00         3.411751e-04   False  False

[32916609 rows x 3 columns]

In [ ]:
brier_score_loss(process(era_1980[1],'total_precipitation')['>25mm'],process(hc_1980_0L[1],'precipitation tp06 [m]')['>25mm'])

In [ ]:
lead='0D'

era_1980 = {mo:pd.read_parquet(era_paths[mo]) for mo in range(1,13)}
hc_1980_0L = {mo:pd.read_parquet(hc_paths[mo]).loc[idx[:,:,:,pd.Timedelta(lead)]] for mo in range(1,13)}

In [ ]:
for mo in range(1,13):
    print(brier_score_loss(process(era_1980[mo],'total_precipitation')['>2.5mm'],process(hc_1980_0L[mo],'precipitation tp06 [m]')['>2.5mm']))

0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516
0.06477903778194516


In [ ]:
brier_score_loss(process(era_1980[1],'total_precipitation')['>25mm'],process(hc_1980_0L[1],'precipitation tp06 [m]')['>25mm'])

0.009891242481578907

In [ ]:
def run_month(lead='0D'):
    losses2p5 = []
    losses25 = []
    for mo in range(1,13):
        era = era_1980[mo]; hc = hc_1980_0L[mo]
        era['mm'] = era['total_precipitation']*1000
        era['>2.5mm'] = era['mm']>2.5
        era['>25mm'] = era['mm']>25

        hc['mm'] = hc['precipitation tp06 [m]']*1000
        hc['>2.5mm'] = hc['mm']>2.5
        hc['>25mm'] = hc['mm']>25
        display(hc['>2.5mm'].values)


        losses2p5 += brier_score_loss(era['>2.5mm'].values,hc['>2.5mm'].values)
        losses25 += brier_score_loss(era['>25mm'].values,hc['>25mm'].values)
        print(losses25)
        print(losses2p5)
    return losses2p5,losses25


In [ ]:
run_month(lead='0D')


array([False, False, False, ..., False, False, False], shape=(715077,))

TypeError: 'float' object is not iterable

In [ ]:
run_month(lead='0D')

total_precipitation        mm  >2.5mm  >25mm
date       lat  lon                                                 
1980-01-01 25.0 235.00             0.000036  0.036219   False  False
                235.25             0.000036  0.035536   False  False
                235.50             0.000056  0.056051   False  False
                235.75             0.000061  0.060840   False  False
                236.00             0.000061  0.060838   False  False
...                                     ...       ...     ...    ...
1980-01-31 49.5 292.00             0.000294  0.294192   False  False
                292.25             0.000437  0.436697   False  False
                292.50             0.000592  0.591902   False  False
                292.75             0.000599  0.598956   False  False
                293.00             0.000648  0.647638   False  False

[715077 rows x 4 columns]